# **Demo**

#### ※ root경로에 'beomi_llama-2-ko-7b_Welssisko-lora_v1.3'폴더를 생성해서 내부에 아래 파일들을 넣어주세요.
  - adapter_model.safetensors
  - adapter_config.json

  해당 파일들은 https://huggingface.co/Ash-Hun/WelSSiSKo_v3_llama-2-ko-base_text-generation 에서 받을 수 있습니다.

In [1]:
# Colab 실행시 경로를 맞춰주세요. Colab에선 T4 GPU를 사용하였습니다.
!cd ./drive/MyDrive/Colab\ Notebooks/my_finetune

/content/drive/MyDrive/Colab Notebooks/my_finetune


In [3]:
!pip install -r requirements.txt

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-n07atzc_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-n07atzc_
  Resolved https://github.com/huggingface/peft.git to commit a1c472f08f39a4b95a228dd436944bc3a75406ea
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 114.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import os

BASE_MODEL_NAME = 'beomi/llama-2-ko-7b'

PEFT_ADAPTOR_NAME = 'beomi_llama-2-ko-7b_Welssisko-lora_v1.3' # 파일 명
PEFT_ADAPTOR_PATH = './beomi_llama-2-ko-7b_Welssisko-lora_v1.3' # 폴더 명
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN") # 본인 HuggingFace Token을 넣어주세요.

In [6]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)
from peft import PeftModel
import warnings
warnings.filterwarnings('ignore')

def prompt_processing(input_text):
    return f'### Instruction:\n{input_text}\n\n### Response:'

def export_output(ouput_text):
    sep = ouput_text[0]['generated_text'].split('### Response:')[1].split('### Instruction')[0].split('## Instruction')[0].split('# Instruction')[0].split('Instruction')[0]
    sep = sep[1:] if sep[0] == '.' else sep
    sep = sep[:sep.find('.')+1] if '.' in sep else sep
    return sep

adapter_name = PEFT_ADAPTOR_NAME

compute_dtype = getattr(torch, 'float16')

lora_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False
)

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_NAME, quantization_config=lora_config, device_map={'': 0}, use_auth_token=HF_TOKEN)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME, trust_remote_code=True, use_auth_token=HF_TOKEN)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = PeftModel.from_pretrained(model, PEFT_ADAPTOR_PATH)

pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer,
                max_length=500,
                do_sample=True,
                temperature=0.1,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                top_k=3,
                # top_p=0.3,
                repetition_penalty = 1.3,
                framework='pt'
                # early_stopping=True
)

config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00015.safetensors:   0%|          | 0.00/919M [00:00<?, ?B/s]

model-00002-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00003-of-00015.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'O

In [7]:
prompt = "장애인 복지와 관련된 세제 혜택을 체크할 때 중요한 점은 무엇인가요?"
result = pipe(prompt_processing(prompt))
print(export_output(result))

 # 장애인에 대한 세금 감면 혜택은 소득세법 제51조의 3항과 같은 법 시행령 제204조에 명시되어 있습니다.


In [8]:
testcase02 = "저희 할머니가 요즘 많이 외로워하셔서, 지역 사회 안에서 어떻게 소통하고 교류할 수 있는지 알아보고 싶어요. 어디로 연락해야 할까요?"
result02 = pipe(prompt_processing(testcase02))
print(export_output(result02))

할머니께서 거주하시는 지역의 노인복지관이나 경로당에서는 다양한 취미 활동 및 친목 도모 프로그램이 마련되어 있습니다.
